# 01 - Sentiment Scoring Pipeline (V2) + Transcript Segmentation

We apply a transformer model (e.g. FinBERT) to each earnings call transcript and perform sentiment features for analysis.

In this version 2 of the sentiment scoring pipeline notebook, we will segment the transcripts into chunks based on when the QA sessions start and also the type of speaker (e.g. analyst vs management). We will then apply the sentiment model to each chunk and aggregate the results.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
torch.cuda.is_available()

Mounted at /content/drive


True

## Imports & configuration
`SAMPLE_SIZE` can be modified and set to small values to perform dry-runs or be set to NOne for a full run.

In [ ]:
from __future__ import annotations

import hashlib
import json
import logging
from pathlib import Path
from typing import Dict, Iterable, List, Optional

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

DATA_ROOT = Path.cwd()
TRANSCRIPTS_PATH = Path('/content/drive/MyDrive/DAI/transcripts.parquet')
CACHE_DIR = Path('/content/drive/MyDrive/DAI/sent_cache')
OUTPUT_DIR = Path('/content/drive/MyDrive/DAI/output')
CHECKPOINT_PATH = OUTPUT_DIR / 'sentiment_features_checkpoint.parquet'

TEXT_COLUMN = 'full_transcript'
ID_COLUMNS = ['symbol', 'year', 'quarter']
SAMPLE_SIZE: Optional[int] = None  # set to None when ready for the full run
BATCH_SIZE = 16
MAX_TOKENS = 450
MIN_TOKENS = 120
OVERLAP_SENTS = 0
WRITE_BATCH_SIZE: Optional[int] = 10000  # set to None to disable checkpoint writing
PREPROC_VERSION = 1

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
CACHE_DIR.mkdir(parents=True, exist_ok=True)
if CHECKPOINT_PATH.exists():
    CHECKPOINT_PATH.unlink()
logging.info('Sentiment pipeline configured.')


## Load transcripts

In [ ]:
if not TRANSCRIPTS_PATH.exists():
    raise FileNotFoundError(f'Transcripts file not found at {TRANSCRIPTS_PATH}. Update TRANSCRIPTS_PATH.')

logging.info('Loading transcripts from %s', TRANSCRIPTS_PATH)
try:
    import pyarrow.parquet as pq
    transcripts_df = pq.read_table(TRANSCRIPTS_PATH).to_pandas()
except Exception as exc:
    logging.warning('Falling back to pandas.read_parquet: %s', exc)
    transcripts_df = pd.read_parquet(TRANSCRIPTS_PATH)

if SAMPLE_SIZE is not None:
    transcripts_df = transcripts_df.head(SAMPLE_SIZE)

logging.info('Loaded %s transcripts', len(transcripts_df))
transcripts_df.head()


,symbol,year,quarter,full_transcript
0,A,2010,1,Operator: Good morning ladies and gentlemen an...
1,A,2010,2,"Operator: Good day, ladies and gentlemen, and ..."
2,A,2011,1,"Operator: Good day, ladies and gentlemen, and ..."
3,A,2011,2,"Operator: Good day, ladies and gentlemen, and ..."
4,A,2011,3,"Operator: Good day, ladies and gentlemen, and ..."


## Sentence segmentation & chunking
We will split the transcripts into sentences and then into chunks of sentences, each consisting of a maximum of 512 tokens, suitable for the model.

since the model (ProsusAI/finbert) we will use are large models that are robust to minor typos and mistakes, we will only perform very minimal preprocessing such as whitespace normalization.

In [ ]:
import spacy

_NLP = spacy.blank('en')
if 'sentencizer' not in _NLP.pipe_names:
    _NLP.add_pipe('sentencizer')

def split_sentences(text: str) -> List[str]:
    if not isinstance(text, str) or not text.strip():
        return []
    doc = _NLP(' '.join(text.split()))
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

def pack_sentences_to_token_chunks(
    sentences: List[str],
    tokenizer,
    max_tokens: int = MAX_TOKENS,
    min_tokens: int = MIN_TOKENS,
    overlap_sents: int = OVERLAP_SENTS,
) -> List[Dict]:
    if not sentences:
        return []
    chunks, buf, buf_tokens = [], [], 0
    i = 0
    while i < len(sentences):
        sent = sentences[i]
        t = len(tokenizer.tokenize(sent))
        if buf and buf_tokens + t > max_tokens:
            chunk_text = ' '.join(buf)
            n_tokens = len(tokenizer.tokenize(chunk_text))
            start_idx = i - len(buf)
            end_idx = i - 1
            chunks.append({
                'text': chunk_text,
                'n_tokens': n_tokens,
                'start_idx': start_idx,
                'end_idx': end_idx,
                'n_sentences': end_idx - start_idx + 1,
            })
            if overlap_sents > 0:
                overlap = buf[-overlap_sents:]
                buf = overlap[:]
                buf_tokens = len(tokenizer.tokenize(' '.join(buf)))
            else:
                buf, buf_tokens = [], 0
            continue
        buf.append(sent)
        buf_tokens += t
        i += 1
    if buf:
        chunk_text = ' '.join(buf)
        n_tokens = len(tokenizer.tokenize(chunk_text))
        start_idx = len(sentences) - len(buf)
        end_idx = len(sentences) - 1
        if chunks and n_tokens < min_tokens:
            prev = chunks.pop()
            merged = prev['text'] + ' ' + chunk_text
            start_idx = prev['start_idx']
            end_idx = len(sentences) - 1
            chunks.append({
                'text': merged,
                'n_tokens': len(tokenizer.tokenize(merged)),
                'start_idx': start_idx,
                'end_idx': end_idx,
                'n_sentences': end_idx - start_idx + 1,
            })
        else:
            chunks.append({
                'text': chunk_text,
                'n_tokens': n_tokens,
                'start_idx': start_idx,
                'end_idx': end_idx,
                'n_sentences': end_idx - start_idx + 1,
            })
    return chunks


In [ ]:
import re
SPEAKER_PATTERN = re.compile(r'^([A-Z][A-Za-z& .()/\'\-]{2,40}):\s*')
QA_CUES = [
    re.compile(r'\bQ&A\b', re.IGNORECASE),
    re.compile(r'question and answer', re.IGNORECASE),
    re.compile(r'we will now begin the question', re.IGNORECASE),
    re.compile(r'your first question comes', re.IGNORECASE),
]
ANALYST_KEYS = ['analyst', 'question comes from', 'q -', 'caller']
MANAGEMENT_KEYS = ['ceo', 'chief', 'cfo', 'coo', 'cto', 'president', 'vp', 'vice president', 'evp', 'svp', 'investor relations', 'ir', 'founder', 'chairman', 'director', 'managing partner']
def normalize_transcript(text: str) -> str:
    if not isinstance(text, str):
        return ''
    # ensure speaker labels start on new lines
    text = re.sub(r'(?<!\n)([A-Z][A-Za-z& .()/\'\-]{2,40}:)', r'\n\1', text)
    return text
def classify_speaker_role(speaker: str) -> str:
    s = (speaker or '').lower()
    if 'operator' in s or 'moderator' in s:
        return 'operator'
    if any(key in s for key in ANALYST_KEYS):
        return 'analyst'
    if any(key in s for key in MANAGEMENT_KEYS):
        return 'mgmt'
    return 'other'
def segment_transcript(text: str) -> List[Dict]:
    normalized = normalize_transcript(text)
    lines = [ln.strip() for ln in normalized.splitlines()]
    segments: List[Dict] = []
    current_section = 'prepared'
    current_segment: Optional[Dict] = None
    for raw in lines:
        if not raw:
            continue
        if any(pattern.search(raw) for pattern in QA_CUES):
            current_section = 'qa'
            if current_segment:
                segments.append(current_segment)
                current_segment = None
            continue
        speaker_match = SPEAKER_PATTERN.match(raw)
        if speaker_match:
            if current_segment:
                segments.append(current_segment)
            speaker = speaker_match.group(1).replace(':', '').strip()
            content = raw[speaker_match.end():].strip()
            role = classify_speaker_role(speaker)
            if current_section == 'prepared' and role == 'analyst':
                current_section = 'qa'
            current_segment = {
                'speaker': speaker,
                'speaker_role': role,
                'section': current_section,
                'text': content,
            }
            continue
        lowered = raw.lower()
        if current_section == 'prepared' and lowered.startswith('your first question'):
            current_section = 'qa'
            if current_segment:
                segments.append(current_segment)
                current_segment = None
            continue
        if current_segment is not None:
            current_segment['text'] = (current_segment['text'] + ' ' + raw).strip()
        else:
            current_segment = {
                'speaker': '',
                'speaker_role': 'other',
                'section': current_section,
                'text': raw,
            }
    if current_segment:
        segments.append(current_segment)
    cleaned = []
    for seg in segments:
        content = seg.get('text', '').strip()
        if not content:
            continue
        if content.startswith('(Operator Instructions'):
            continue
        seg['text'] = content
        cleaned.append(seg)
    if not cleaned:
        cleaned = [{'speaker': '', 'speaker_role': 'other', 'section': 'prepared', 'text': text or ''}]
    return cleaned


## Load FinBERT (or another Hugging Face classifier)
The default is `ProsusAI/finbert`.

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = 'ProsusAI/finbert'
# MODEL_NAME = 'j-hartmann/emotion-english-distilroberta-base'
# MODEL_NAME = 'yiyanghkust/finbert-tone'

SUFFIX = '_finbert' # modify the suffix to avoid overwriting previous results


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
else:
    DEVICE = torch.device('cpu')

logging.info('Using device: %s', DEVICE)

model.to(DEVICE)
model.eval()
LABEL_ORDER = [model.config.id2label[i] for i in range(model.config.num_labels)]
LABEL_ORDER


/Users/aryahassibi/Desktop/School/Data and Ai in economics/jupy/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

## Scoring the transcripts


In [ ]:
def load_cache(cache_dir: Path = CACHE_DIR) -> Path:
    cache_dir.mkdir(exist_ok=True)
    return cache_dir

def stable_hash(text: str) -> str:
    return hashlib.blake2s(text.encode('utf-8'), digest_size=8).hexdigest()

def maybe_read_cache(cache_dir: Path, key: str):
    fp = cache_dir / f'{key}.json'
    if fp.exists():
        return json.loads(fp.read_text())
    return None

def write_cache(cache_dir: Path, key: str, payload):
    (cache_dir / f'{key}.json').write_text(json.dumps(payload))

@torch.inference_mode()
def score_chunks(
    chunks: List[Dict],
    tokenizer=tokenizer,
    model=model,
    device=DEVICE,
    cache_dir: Path = CACHE_DIR,
    batch_size: int = BATCH_SIZE,
    max_length: int = 512,
    label_order: Iterable[str] = LABEL_ORDER,
) -> List[Dict]:
    cache_dir = load_cache(cache_dir)
    results = []
    batch_texts, batch_keys, pending = [], [], []

    def flush():
        if not batch_texts:
            return
        enc = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors='pt',
        )
        enc = {k: v.to(device) for k, v in enc.items()}
        logits = model(**enc).logits
        probs = F.softmax(logits, dim=-1).cpu().tolist()
        formatted = [
            [{
                'label': label_order[j],
                'score': row[j],
            } for j in range(len(label_order))]
            for row in probs
        ]
        for key, payload, meta in zip(batch_keys, formatted, pending):
            # write_cache(cache_dir, key, payload)
            results.append({
                'section': meta.get('section', 'prepared'),
                'speaker_role': meta.get('speaker_role', 'other'),
                'n_sentences': meta.get('n_sentences', 0),
                'key': key,
                'scores': payload,
                'n_tokens': meta['n_tokens'],
            })
        batch_texts.clear()
        batch_keys.clear()
        pending.clear()

    for ch in chunks:
        key = stable_hash(ch['text'])
        # !!!cashing disabled for google collab cause it doesn't make sense
        # cached = maybe_read_cache(cache_dir, key)
        # if cached is not None:
        #     results.append({
                # 'section': meta.get('section', 'prepared'),
                # 'speaker_role': meta.get('speaker_role', 'other'),
                # 'n_sentences': meta.get('n_sentences', 0),
        #         'key': key,
        #         'scores': cached,
        #         'n_tokens': ch['n_tokens'],
        #     })
        #     continue
        batch_texts.append(ch['text'])
        batch_keys.append(key)
        pending.append(ch)
        if len(batch_texts) >= batch_size:
            flush()
    flush()
    return results


## Aggregate chunk scores to transcript-level features


In [ ]:
def distlist_to_array(distlist: List[Dict], label_order: Iterable[str] = LABEL_ORDER):
    labels = [label.lower() for label in label_order]
    if not distlist:
        return np.empty((0, len(labels))), np.empty((0,)), np.empty((0,)), labels
    probs, weights, sent_counts = [], [], []
    for item in distlist:
        lookup = {d['label'].lower(): d['score'] for d in item['scores']}
        probs.append([lookup.get(label, 0.0) for label in labels])
        weights.append(item.get('n_tokens', 0))
        sent_counts.append(item.get('n_sentences', 0))
    return np.array(probs), np.array(weights), np.array(sent_counts), labels

def aggregate_subset(distlist: List[Dict], labels: List[str], prefix: str = '') -> Dict[str, float]:
    stats: Dict[str, float] = {}
    if not distlist:
        for label in labels:
            stats[f'{prefix}p_{label}_wmean'] = float('nan')
            stats[f'{prefix}p_{label}_std'] = float('nan')
            stats[f'{prefix}p_{label}_q95'] = float('nan')
            stats[f'{prefix}p_{label}_q05'] = float('nan')
            stats[f'{prefix}share_{label}_gt70'] = float('nan')
        stats[f'{prefix}entropy_wmean'] = float('nan')
        stats[f'{prefix}n_chunks'] = 0
        stats[f'{prefix}n_tokens_total'] = 0.0
        stats[f'{prefix}n_sentences'] = 0
        stats[f'{prefix}max_positive'] = float('nan')
        stats[f'{prefix}max_negative'] = float('nan')
        return stats
    probs, weights, sent_counts, labels = distlist_to_array(distlist, labels)
    total_tokens = weights.sum()
    if total_tokens > 0:
        w = weights / total_tokens
    else:
        w = np.ones_like(weights) / len(weights)
    for idx, label in enumerate(labels):
        pj = probs[:, idx]
        stats[f'{prefix}p_{label}_wmean'] = float(np.dot(pj, w))
        stats[f'{prefix}p_{label}_std'] = float(pj.std())
        stats[f'{prefix}p_{label}_q95'] = float(np.quantile(pj, 0.95))
        stats[f'{prefix}p_{label}_q05'] = float(np.quantile(pj, 0.05))
        stats[f'{prefix}share_{label}_gt70'] = float((pj > 0.70).mean())
    entropy = -(probs * np.log(probs + 1e-12)).sum(axis=1)
    stats[f'{prefix}entropy_wmean'] = float(np.dot(entropy, w))
    stats[f'{prefix}n_chunks'] = int(len(distlist))
    stats[f'{prefix}n_tokens_total'] = float(total_tokens)
    stats[f'{prefix}n_sentences'] = int(sent_counts.sum())
    label_map = {label: idx for idx, label in enumerate(labels)}
    stats[f'{prefix}max_positive'] = float(probs[:, label_map['positive']].max()) if 'positive' in label_map else float('nan')
    stats[f'{prefix}max_negative'] = float(probs[:, label_map['negative']].max()) if 'negative' in label_map else float('nan')
    return stats

def aggregate_sentiment(distlist: List[Dict], label_order: Iterable[str] = LABEL_ORDER) -> Dict[str, float]:
    labels = [label.lower() for label in label_order]
    result = dict(aggregate_subset(distlist, labels, prefix=''))
    subsets = {
        'prepared_': [item for item in distlist if item.get('section') == 'prepared'],
        'qa_': [item for item in distlist if item.get('section') == 'qa'],
        'mgmt_': [item for item in distlist if item.get('speaker_role') == 'mgmt'],
        'analyst_': [item for item in distlist if item.get('speaker_role') == 'analyst'],
    }
    for prefix, subset in subsets.items():
        result.update(aggregate_subset(subset, labels, prefix=prefix))
    if 'p_positive_wmean' in result and 'p_negative_wmean' in result:
        result['net_pos'] = result['p_positive_wmean'] - result['p_negative_wmean']
    for prefix in ['prepared_', 'qa_', 'mgmt_', 'analyst_']:
        pos_key = f'{prefix}p_positive_wmean'
        neg_key = f'{prefix}p_negative_wmean'
        if pos_key in result and neg_key in result:
            result[f'{prefix}net_pos'] = result[pos_key] - result[neg_key]
    if 'prepared_net_pos' in result and 'qa_net_pos' in result:
        result['delta_prepared_qa_net_pos'] = result['prepared_net_pos'] - result['qa_net_pos']
    if 'mgmt_net_pos' in result and 'analyst_net_pos' in result:
        result['delta_mgmt_analyst_net_pos'] = result['mgmt_net_pos'] - result['analyst_net_pos']
    return result


## Transcript-level wrapper


In [ ]:
def suffix_features(features: Dict[str, float], suffix: str) -> Dict[str, float]:
    return {f'{key}{suffix}': value for key, value in features.items()}

def score_transcript(text: str) -> Dict[str, float]:
    segments = segment_transcript(text)
    all_chunks: List[Dict] = []
    total_sentences = 0
    for segment in segments:
        sentences = split_sentences(segment['text'])
        total_sentences += len(sentences)
        segment_chunks = pack_sentences_to_token_chunks(sentences, tokenizer)
        for chunk in segment_chunks:
            chunk['section'] = segment.get('section', 'prepared')
            chunk['speaker_role'] = segment.get('speaker_role', 'other')
            chunk['n_sentences'] = chunk.get('n_sentences', chunk['end_idx'] - chunk['start_idx'] + 1)
            all_chunks.append(chunk)
    if not all_chunks:
        sentences = split_sentences(text)
        total_sentences = len(sentences)
        fallback_chunks = pack_sentences_to_token_chunks(sentences, tokenizer)
        for chunk in fallback_chunks:
            chunk['section'] = 'prepared'
            chunk['speaker_role'] = 'other'
            chunk['n_sentences'] = chunk.get('n_sentences', chunk['end_idx'] - chunk['start_idx'] + 1)
        all_chunks = fallback_chunks
    distributions = score_chunks(all_chunks) if all_chunks else []
    for dist, chunk in zip(distributions, all_chunks):
        dist['section'] = chunk.get('section', 'prepared')
        dist['speaker_role'] = chunk.get('speaker_role', 'other')
        dist['n_sentences'] = chunk.get('n_sentences', chunk.get('end_idx', 0) - chunk.get('start_idx', -1))
        dist['n_tokens'] = chunk.get('n_tokens', 0)
    features = aggregate_sentiment(distributions)
    suffixed = suffix_features(features, SUFFIX)
    suffixed['total_sentences' + SUFFIX] = total_sentences
    return suffixed

def score_dataframe(
    df: pd.DataFrame,
    text_column: str,
    id_columns: List[str],
    write_batch_size: Optional[int] = None,
    checkpoint_path: Optional[Path] = None,
) -> pd.DataFrame:
    id_columns = [col for col in id_columns if col in df.columns]
    records: List[Dict] = []
    batch_rows: List[Dict] = []
    writer = None
    if write_batch_size and checkpoint_path and checkpoint_path.exists():
        checkpoint_path.unlink()
    for row in tqdm(df.itertuples(index=False), total=len(df)):
        payload = {col: getattr(row, col) for col in id_columns}
        text = getattr(row, text_column)
        payload.update(score_transcript(text))
        records.append(payload)
        if write_batch_size and checkpoint_path:
            batch_rows.append(payload)
            if len(batch_rows) >= write_batch_size:
                import pyarrow as pa
                import pyarrow.parquet as pq
                df_batch = pd.DataFrame(batch_rows)
                table = pa.Table.from_pandas(df_batch)
                if writer is None:
                    writer = pq.ParquetWriter(checkpoint_path, table.schema)
                writer.write_table(table)
                batch_rows.clear()
    if write_batch_size and checkpoint_path and batch_rows:
        import pyarrow as pa
        import pyarrow.parquet as pq
        df_batch = pd.DataFrame(batch_rows)
        table = pa.Table.from_pandas(df_batch)
        if writer is None:
            writer = pq.ParquetWriter(checkpoint_path, table.schema)
        writer.write_table(table)
        batch_rows.clear()
    if writer is not None:
        writer.close()
    return pd.DataFrame(records)


## Run the sentiment pipeline


In [ ]:
sentiment_df = score_dataframe(
    transcripts_df,
    TEXT_COLUMN,
    ID_COLUMNS,
    write_batch_size=WRITE_BATCH_SIZE,
    checkpoint_path=CHECKPOINT_PATH,
)
logging.info('Generated sentiment features for %s transcripts', len(sentiment_df))
sentiment_df.head()


  0%|          | 0/77304 [00:00<?, ?it/s]

## Save features to disk


In [ ]:
sentiment_df.to_parquet(OUTPUT_DIR / f"sentiment_features{SUFFIX}.parquet", index=False)
logging.info('Wrote sentiment features to %s', OUTPUT_DIR / f"sentiment_features{SUFFIX}.parquet")
sentiment_df.to_csv(OUTPUT_DIR / f"sentiment_features{SUFFIX}.csv", index=False)
logging.info('Also exported CSV for quick inspection.')


## Feature sanity checks


In [ ]:
prob_cols = [col for col in sentiment_df.columns if col.startswith('p_')]
sentiment_df[prob_cols].describe().T

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 4))
col_name = 'p_positive_wmean'
sentiment_df['p_positive_wmean' + SUFFIX].hist(bins=30, alpha=0.7, ax=ax)

ax.set_title('Distribution of weighted positive probabilities')
plt.tight_layout()
